<font color='black'>Q1. (CNN with	gender	classification	32x32	image)</font>

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.local import LocallyConnected2D

Using TensorFlow backend.


In [2]:
num_classes = 2

directory_32 = "data/womanman-32/"
directory_man = "man/"
directory_woman = "woman/"

In [3]:
filelist_32_man = [file for file in os.listdir(directory_32 + directory_man) if file.endswith('.png')]
filelist_32_woman = [file for file in os.listdir(directory_32 + directory_woman) if file.endswith('.png')]

In [4]:
X = np.zeros([len(filelist_32_man) + len(filelist_32_woman), 32, 32, 3])
Y = np.zeros([len(filelist_32_man) + len(filelist_32_woman), 2])

index = 0
for file in filelist_32_man:
    X[index] = np.array(Image.open(directory_32 + directory_man + file))
    Y[index][0] = 1
    index = index + 1
for file in filelist_32_woman:
    X[index] = np.array(Image.open(directory_32 + directory_woman + file))
    Y[index][1] = 1
    index = index + 1

In [5]:
randomShuffle = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)

In [22]:
def createCNNModel(n_classes):
    model = Sequential()
    # Convenlutional layer 1
    model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3), activation='relu'))
    # Pooling layer 1
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convenlutional layer 2
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    # Pooling layer 2
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(Dropout(0.05))
    # Convenlutional layer 3
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    # Pooling layer 3
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    # Locally-connected layer
    #model.add(LocallyConnected2D(256, (2, 2)))
    
    model.add(Flatten())
    # Fully-connected layer 1
    model.add(Dense(256, activation='relu'))
    # Fully-connected layer 2
    model.add(Dense(n_classes, activation='softmax'))
    epochs = 75
    optimizer_adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer_adam, metrics=['accuracy'])
    print(model.summary())
    return model, epochs

In [23]:
accuracies = []

In [24]:
for train_index, test_index in randomShuffle.split(X):
    
    X_train = X[train_index, :]
    Y_train = Y[train_index, :]

    X_test = X[test_index, :]
    Y_test = Y[test_index, :]
    
    model, epochs = createCNNModel(num_classes)
    
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=1, batch_size=100)
    scores = model.evaluate(X_test, Y_test)
    accuracies.append(scores[1]*100)
    

C:\Users\vwslz\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\Users\vwslz\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
C:\Users\vwslz\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  del sys.path[0]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_121 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_121 (MaxPoolin (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_81 (Dropout)         (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_123 (MaxPoolin (None, 2, 2, 128)         0         
__________

1124/1124 [==============================] - 0s 335us/step - loss: 0.6182 - acc: 0.7117 - val_loss: 0.6336 - val_acc: 0.6879
Epoch 19/75
1124/1124 [==============================] - 0s 295us/step - loss: 0.6095 - acc: 0.7367 - val_loss: 0.6116 - val_acc: 0.7021
Epoch 20/75
1124/1124 [==============================] - 0s 288us/step - loss: 0.5520 - acc: 0.7402 - val_loss: 0.5983 - val_acc: 0.6986
Epoch 21/75
1124/1124 [==============================] - 0s 308us/step - loss: 0.5442 - acc: 0.7562 - val_loss: 0.6014 - val_acc: 0.6986
Epoch 22/75
1124/1124 [==============================] - 0s 312us/step - loss: 0.5104 - acc: 0.7642 - val_loss: 0.5948 - val_acc: 0.7270
Epoch 23/75
1124/1124 [==============================] - 0s 306us/step - loss: 0.5204 - acc: 0.7651 - val_loss: 0.6148 - val_acc: 0.7234
Epoch 24/75
1124/1124 [==============================] - 0s 314us/step - loss: 0.4740 - acc: 0.7669 - val_loss: 0.6380 - val_acc: 0.7092
Epoch 25/75
1124/1124 [==============================

dense_86 (Dense)             (None, 2)                 514       
Total params: 225,090
Trainable params: 225,090
Non-trainable params: 0
_________________________________________________________________
None
Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 3s 3ms/step - loss: 6.4989 - acc: 0.4911 - val_loss: 4.6817 - val_acc: 0.4858
Epoch 2/75
1124/1124 [==============================] - 0s 309us/step - loss: 5.2505 - acc: 0.5044 - val_loss: 2.4085 - val_acc: 0.5922
Epoch 3/75
1124/1124 [==============================] - 0s 315us/step - loss: 4.0828 - acc: 0.5391 - val_loss: 1.9331 - val_acc: 0.5638
Epoch 4/75
1124/1124 [==============================] - 0s 311us/step - loss: 2.8789 - acc: 0.5783 - val_loss: 1.9856 - val_acc: 0.5851
Epoch 5/75
1124/1124 [==============================] - 0s 302us/step - loss: 2.6089 - acc: 0.5845 - val_loss: 1.2762 - val_acc: 0.6312
Epoch 6/75
1124/1124 [==============================] - 0s 324us/s

1124/1124 [==============================] - 0s 313us/step - loss: 0.5259 - acc: 0.7598 - val_loss: 0.6589 - val_acc: 0.6915
Epoch 30/75
1124/1124 [==============================] - 0s 310us/step - loss: 0.5183 - acc: 0.7625 - val_loss: 0.6614 - val_acc: 0.6986
Epoch 31/75
1124/1124 [==============================] - 0s 298us/step - loss: 0.4728 - acc: 0.7847 - val_loss: 0.6507 - val_acc: 0.6879
Epoch 32/75
1124/1124 [==============================] - 0s 291us/step - loss: 0.4497 - acc: 0.7963 - val_loss: 0.6485 - val_acc: 0.6879
Epoch 33/75
1124/1124 [==============================] - 0s 314us/step - loss: 0.4528 - acc: 0.7847 - val_loss: 0.6666 - val_acc: 0.6950
Epoch 34/75
1124/1124 [==============================] - 0s 314us/step - loss: 0.4734 - acc: 0.7927 - val_loss: 0.6612 - val_acc: 0.6915
Epoch 35/75
1124/1124 [==============================] - 0s 307us/step - loss: 0.4717 - acc: 0.7811 - val_loss: 0.6677 - val_acc: 0.7163
Epoch 36/75
1124/1124 [==============================

...

Epoch 2/75
1124/1124 [==============================] - 0s 308us/step - loss: 3.1497 - acc: 0.5427 - val_loss: 1.3055 - val_acc: 0.5957
Epoch 3/75
1124/1124 [==============================] - 0s 313us/step - loss: 2.7624 - acc: 0.5302 - val_loss: 1.0628 - val_acc: 0.6312
Epoch 4/75
1124/1124 [==============================] - 0s 320us/step - loss: 2.2590 - acc: 0.5472 - val_loss: 0.9921 - val_acc: 0.6596
Epoch 5/75
1124/1124 [==============================] - 0s 317us/step - loss: 1.6882 - acc: 0.6077 - val_loss: 0.8810 - val_acc: 0.6844
Epoch 6/75
1124/1124 [==============================] - 0s 311us/step - loss: 1.5028 - acc: 0.6085 - val_loss: 0.8342 - val_acc: 0.7092
Epoch 7/75
1124/1124 [==============================] - 0s 309us/step - loss: 1.3055 - acc: 0.6192 - val_loss: 0.7903 - val_acc: 0.7234
Epoch 8/75
1124/1124 [==============================] - 0s 322us/step - loss: 1.1630 - acc: 0.6352 - val_loss: 0.7719 - val_acc: 0.7270
Epoch 9/75
1124/1124 [==========================

Epoch 62/75
1124/1124 [==============================] - 0s 313us/step - loss: 0.1295 - acc: 0.9520 - val_loss: 0.6412 - val_acc: 0.7730
Epoch 63/75
1124/1124 [==============================] - 0s 320us/step - loss: 0.1171 - acc: 0.9582 - val_loss: 0.6001 - val_acc: 0.7872
Epoch 64/75
1124/1124 [==============================] - 0s 321us/step - loss: 0.0973 - acc: 0.9715 - val_loss: 0.6366 - val_acc: 0.7660
Epoch 65/75
1124/1124 [==============================] - 0s 323us/step - loss: 0.1114 - acc: 0.9635 - val_loss: 0.6351 - val_acc: 0.7730
Epoch 66/75
1124/1124 [==============================] - 0s 318us/step - loss: 0.1100 - acc: 0.9573 - val_loss: 0.6159 - val_acc: 0.7872
Epoch 67/75
1124/1124 [==============================] - 0s 290us/step - loss: 0.1094 - acc: 0.9662 - val_loss: 0.6214 - val_acc: 0.7908
Epoch 68/75
1124/1124 [==============================] - 0s 297us/step - loss: 0.1107 - acc: 0.9546 - val_loss: 0.6147 - val_acc: 0.8014
Epoch 69/75
1124/1124 [==================

1124/1124 [==============================] - 0s 322us/step - loss: 0.5192 - acc: 0.7731 - val_loss: 0.6655 - val_acc: 0.7411
Epoch 34/75
1124/1124 [==============================] - 0s 320us/step - loss: 0.4865 - acc: 0.7847 - val_loss: 0.6663 - val_acc: 0.7199
Epoch 35/75
1124/1124 [==============================] - 0s 310us/step - loss: 0.4564 - acc: 0.7856 - val_loss: 0.6502 - val_acc: 0.7057
Epoch 36/75
1124/1124 [==============================] - 0s 302us/step - loss: 0.4663 - acc: 0.7972 - val_loss: 0.6686 - val_acc: 0.7163
Epoch 37/75
1124/1124 [==============================] - 0s 325us/step - loss: 0.4343 - acc: 0.7998 - val_loss: 0.6754 - val_acc: 0.7234
Epoch 38/75
1124/1124 [==============================] - 0s 317us/step - loss: 0.4138 - acc: 0.8052 - val_loss: 0.6784 - val_acc: 0.7092
Epoch 39/75
1124/1124 [==============================] - 0s 285us/step - loss: 0.4266 - acc: 0.8034 - val_loss: 0.6844 - val_acc: 0.7163
Epoch 40/75
1124/1124 [==============================

1124/1124 [==============================] - 0s 318us/step - loss: 2.0718 - acc: 0.5765 - val_loss: 1.3031 - val_acc: 0.6028
Epoch 5/75
1124/1124 [==============================] - 0s 313us/step - loss: 1.6942 - acc: 0.6005 - val_loss: 1.2116 - val_acc: 0.6206
Epoch 6/75
1124/1124 [==============================] - 0s 305us/step - loss: 1.5062 - acc: 0.6103 - val_loss: 1.1237 - val_acc: 0.5887
Epoch 7/75
1124/1124 [==============================] - 0s 316us/step - loss: 1.3558 - acc: 0.6032 - val_loss: 1.0633 - val_acc: 0.5993
Epoch 8/75
1124/1124 [==============================] - 0s 322us/step - loss: 1.0946 - acc: 0.6495 - val_loss: 0.9187 - val_acc: 0.6206
Epoch 9/75
1124/1124 [==============================] - 0s 335us/step - loss: 1.0477 - acc: 0.6477 - val_loss: 0.8827 - val_acc: 0.5993
Epoch 10/75
1124/1124 [==============================] - 0s 309us/step - loss: 0.9837 - acc: 0.6539 - val_loss: 0.8551 - val_acc: 0.6206
Epoch 11/75
1124/1124 [==============================] - 0

Epoch 34/75
1124/1124 [==============================] - 0s 322us/step - loss: 0.4160 - acc: 0.8078 - val_loss: 0.6736 - val_acc: 0.7128
Epoch 35/75
1124/1124 [==============================] - 0s 317us/step - loss: 0.3798 - acc: 0.8176 - val_loss: 0.6660 - val_acc: 0.6986
Epoch 36/75
1124/1124 [==============================] - 0s 317us/step - loss: 0.4010 - acc: 0.8327 - val_loss: 0.6682 - val_acc: 0.7128
Epoch 37/75
1124/1124 [==============================] - 0s 314us/step - loss: 0.4186 - acc: 0.8132 - val_loss: 0.6705 - val_acc: 0.7128
Epoch 38/75
1124/1124 [==============================] - 0s 310us/step - loss: 0.4009 - acc: 0.8247 - val_loss: 0.6782 - val_acc: 0.7092
Epoch 39/75
1124/1124 [==============================] - 0s 331us/step - loss: 0.3826 - acc: 0.8319 - val_loss: 0.6840 - val_acc: 0.7199
Epoch 40/75
1124/1124 [==============================] - 0s 315us/step - loss: 0.3785 - acc: 0.8238 - val_loss: 0.6757 - val_acc: 0.7057
Epoch 41/75
1124/1124 [==================

In [25]:
print(accuracies)

[75.17730487999341, 78.36879428396834, 69.85815615518719, 74.11347509275937, 74.8226948663698, 75.53191472452583, 79.78723383118921, 76.24113492086424, 73.40425519233055, 74.46808523320138]


In [26]:
avg = sum(accuracies) / 10
print(avg)

75.17730491803893


<font color='black'>Q2. (CNN with	gender	classification	256x256	image)</font>

In [1]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.local import LocallyConnected2D
from keras.utils import np_utils
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [2]:
num_classes = 2

directory_256 = "data/womanman-256/"
directory_man = "man/"
directory_woman = "woman/"

filelist_256_man = [file for file in os.listdir(directory_256 + directory_man) if file.endswith('.png')]
filelist_256_woman = [file for file in os.listdir(directory_256 + directory_woman) if file.endswith('.png')]

randomShuffle = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)

X = np.zeros([len(filelist_256_man) + len(filelist_256_woman), 256, 256, 3])
Y = np.zeros([len(filelist_256_man) + len(filelist_256_woman), 1])

accuracies = []

In [3]:
index = 0
for file in filelist_256_man:
    X[index] = np.array(Image.open(directory_256 + directory_man + file))
    Y[index] = 0
    index = index + 1
for file in filelist_256_woman:
    X[index] = np.array(Image.open(directory_256 + directory_woman + file))
    Y[index] = 1
    index = index + 1

In [4]:
Y = np_utils.to_categorical(Y, 2)

In [12]:
def createCNNModel(n_classes):
    model = Sequential()
    
    # Convenlutional layer 1
    model.add(Conv2D(16, (8, 8), input_shape=(256, 256, 3), activation='relu', strides=(4, 4)))
    # Pooling layer 1
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    # Convenlutional layer 2
    model.add(Conv2D(64, (4, 4), activation='relu', padding='same', strides=(2, 2)))
    # Pooling layer 2
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    # Convenlutional layer 3
    model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
    # Pooling layer 3
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    # Locally-connected layer
    #model.add(LocallyConnected2D(256, (2, 2)))
    
    model.add(Flatten())
    
    model.add(Dropout(0.5))
    
    # Fully-connected layer 1
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
    
    # Fully-connected layer 2
    model.add(Dense(n_classes, activation='softmax'))
    
    epochs = 75
    
    #optimizer_adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model, epochs

In [13]:
for train_index, test_index in randomShuffle.split(X):
    
    X_train = X[train_index, :]
    Y_train = Y[train_index, :]

    X_test = X[test_index, :]
    Y_test = Y[test_index, :]
    
    model, epochs = createCNNModel(num_classes)
    
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, verbose=1, batch_size=64)
    scores = model.evaluate(X_test, Y_test)
    print(scores[1] * 100)
    accuracies.append(scores[1]*100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 63, 63, 16)        3088      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 31, 31, 16)        0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 31, 31, 16)        64        
_________________________________________________________________
dropout_61 (Dropout)         (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 16, 16, 64)        16448     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_50 (Batc (None, 8, 8, 64)          256       
__________

Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7150 - acc: 0.6299 - val_loss: 0.6392 - val_acc: 0.6560
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6756 - acc: 0.6664 - val_loss: 0.6007 - val_acc: 0.6560
Epoch 8/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6693 - acc: 0.6468 - val_loss: 0.5797 - val_acc: 0.6809
Epoch 9/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6319 - acc: 0.6931 - val_loss: 0.8100 - val_acc: 0.5603
Epoch 10/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5992 - acc: 0.7206 - val_loss: 0.8080 - val_acc: 0.5851
Epoch 11/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5892 - acc: 0.7171 - val_loss: 0.6395 - val_acc: 0.6560
Epoch 12/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5611 - acc: 0.7224 - val_loss: 0.5982 - val_acc: 0.6915
Epoch 13/75
1124/1124 [==============================] - 3s

Epoch 67/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.2081 - acc: 0.9270 - val_loss: 0.4599 - val_acc: 0.8333
Epoch 68/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.1925 - acc: 0.9110 - val_loss: 0.4793 - val_acc: 0.8156
Epoch 69/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.1615 - acc: 0.9342 - val_loss: 0.5868 - val_acc: 0.7730
Epoch 70/75
1124/1124 [==============================] - 4s 3ms/step - loss: 0.1857 - acc: 0.9262 - val_loss: 0.5067 - val_acc: 0.8085
Epoch 71/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.1820 - acc: 0.9226 - val_loss: 0.5105 - val_acc: 0.8050
Epoch 72/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.1694 - acc: 0.9306 - val_loss: 0.4541 - val_acc: 0.8191
Epoch 73/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.1679 - acc: 0.9422 - val_loss: 0.5316 - val_acc: 0.7943
Epoch 74/75
1124/1124 [==============================] 

Epoch 32/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3749 - acc: 0.8319 - val_loss: 0.7295 - val_acc: 0.6915
Epoch 33/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3688 - acc: 0.8238 - val_loss: 0.5935 - val_acc: 0.7234
Epoch 34/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3689 - acc: 0.8434 - val_loss: 0.8309 - val_acc: 0.6667
Epoch 35/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3839 - acc: 0.8327 - val_loss: 0.6595 - val_acc: 0.6950
Epoch 36/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3633 - acc: 0.8416 - val_loss: 0.6992 - val_acc: 0.6844
Epoch 37/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3389 - acc: 0.8559 - val_loss: 0.5929 - val_acc: 0.7376
Epoch 38/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3223 - acc: 0.8710 - val_loss: 0.5427 - val_acc: 0.7518
Epoch 39/75
1124/1124 [==============================] 

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 6s 6ms/step - loss: 0.9570 - acc: 0.5133 - val_loss: 0.7084 - val_acc: 0.5957
Epoch 2/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8618 - acc: 0.5596 - val_loss: 0.6585 - val_acc: 0.6206
Epoch 3/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7811 - acc: 0.5819 - val_loss: 0.6279 - val_acc: 0.6312
Epoch 4/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7091 - acc: 0.6246 - val_loss: 0.6141 - val_acc: 0.6667
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6717 - acc: 0.6415 - val_loss: 0.6038 - val_acc: 0.6702
Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6654 - acc: 0.6566 - val_loss: 0.6199 - val_acc: 0.6667
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6654 - acc: 0.6566 - val_loss: 0.5831 - val_acc: 0.7057
Epoch 8/75
1124

...

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 6s 6ms/step - loss: 1.0297 - acc: 0.5089 - val_loss: 0.7073 - val_acc: 0.5567
Epoch 2/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8615 - acc: 0.5578 - val_loss: 0.6780 - val_acc: 0.6312
Epoch 3/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7962 - acc: 0.5854 - val_loss: 0.6161 - val_acc: 0.6596
Epoch 4/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7250 - acc: 0.6281 - val_loss: 0.5860 - val_acc: 0.6773
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7022 - acc: 0.6272 - val_loss: 0.5978 - val_acc: 0.7021
Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6673 - acc: 0.6619 - val_loss: 0.6116 - val_acc: 0.6241
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6396 - acc: 0.6833 - val_loss: 0.5576 - val_acc: 0.7199
Epoch 8/75
1124

Epoch 26/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3737 - acc: 0.8230 - val_loss: 0.6447 - val_acc: 0.7057
Epoch 27/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3516 - acc: 0.8407 - val_loss: 0.7118 - val_acc: 0.6844
Epoch 28/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3820 - acc: 0.8399 - val_loss: 0.6052 - val_acc: 0.6809
Epoch 29/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3429 - acc: 0.8416 - val_loss: 0.5322 - val_acc: 0.7801
Epoch 30/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3804 - acc: 0.8230 - val_loss: 0.7182 - val_acc: 0.6986
Epoch 31/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3550 - acc: 0.8363 - val_loss: 0.5519 - val_acc: 0.7376
Epoch 32/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3386 - acc: 0.8612 - val_loss: 0.5365 - val_acc: 0.7447
Epoch 33/75
1124/1124 [==============================] 

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 8s 7ms/step - loss: 1.1252 - acc: 0.5329 - val_loss: 0.7554 - val_acc: 0.5957
Epoch 2/75
1124/1124 [==============================] - 4s 3ms/step - loss: 0.8549 - acc: 0.5765 - val_loss: 0.7624 - val_acc: 0.6241
Epoch 3/75
1124/1124 [==============================] - 4s 3ms/step - loss: 0.8425 - acc: 0.6005 - val_loss: 0.6194 - val_acc: 0.7234
Epoch 4/75
1124/1124 [==============================] - 4s 3ms/step - loss: 0.8026 - acc: 0.6192 - val_loss: 0.6129 - val_acc: 0.7057
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7527 - acc: 0.6343 - val_loss: 0.5801 - val_acc: 0.7199
Epoch 6/75
1124/1124 [==============================] - 4s 4ms/step - loss: 0.6920 - acc: 0.6690 - val_loss: 1.1028 - val_acc: 0.5106
Epoch 7/75
1124/1124 [==============================] - 4s 4ms/step - loss: 0.6851 - acc: 0.6601 - val_loss: 0.7418 - val_acc: 0.5780
Epoch 8/75
1124

Epoch 26/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.4036 - acc: 0.7998 - val_loss: 0.6645 - val_acc: 0.7270
Epoch 27/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3772 - acc: 0.8274 - val_loss: 0.5436 - val_acc: 0.7660
Epoch 28/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3724 - acc: 0.8390 - val_loss: 0.6313 - val_acc: 0.7199
Epoch 29/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3427 - acc: 0.8461 - val_loss: 0.6538 - val_acc: 0.7057
Epoch 30/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3672 - acc: 0.8363 - val_loss: 0.6135 - val_acc: 0.7518
Epoch 31/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3332 - acc: 0.8594 - val_loss: 0.6467 - val_acc: 0.7128
Epoch 32/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3313 - acc: 0.8550 - val_loss: 0.5662 - val_acc: 0.7589
Epoch 33/75
1124/1124 [==============================] 

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 8s 7ms/step - loss: 1.0554 - acc: 0.5214 - val_loss: 0.7512 - val_acc: 0.5390
Epoch 2/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8536 - acc: 0.5614 - val_loss: 0.6732 - val_acc: 0.6312
Epoch 3/75
1124/1124 [==============================] - 4s 3ms/step - loss: 0.7641 - acc: 0.6174 - val_loss: 0.6686 - val_acc: 0.6383
Epoch 4/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7941 - acc: 0.5996 - val_loss: 0.8133 - val_acc: 0.6099
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7515 - acc: 0.6121 - val_loss: 1.6297 - val_acc: 0.4894
Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.7237 - acc: 0.6361 - val_loss: 0.8331 - val_acc: 0.6170
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6553 - acc: 0.6655 - val_loss: 0.6320 - val_acc: 0.6560
Epoch 8/75
1124

In [22]:
print(accuracies)

[82.97872353107371, 82.2695033770081, 72.69503533417452, 78.72340433986474, 75.88652490724063, 79.07801401530598, 78.72340408622796, 81.91489370156687, 80.14184405617681, 79.78723416937157]


In [24]:
avg = sum(accuracies) / 10
print(avg)

79.21985815180109


<font color='black'>Q3. (LeNet)</font>

In [36]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import OneHotEncoder
import os
from PIL import Image

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.local import LocallyConnected2D

from keras.utils import np_utils
from keras.layers import BatchNormalization

# from tensorflow.examples.tutorials.mnist import input_data
# import random
# from sklearn.utils import shuffle

# from keras.datasets import mnist

Q3.

In [37]:
# mnist = input_data.read_data_sets("MNIST_data/", reshape=False)
# X_train, y_train           = mnist.train.images, mnist.train.labels
# X_validation, y_validation = mnist.validation.images, mnist.validation.labels
# X_test, y_test             = mnist.test.images, mnist.test.labels

# assert(len(X_train) == len(y_train))
# assert(len(X_validation) == len(y_validation))
# assert(len(X_test) == len(y_test))

# print()
# print("Image Shape: {}".format(X_train[0].shape))
# print()
# print("Training Set:   {} samples".format(len(X_train)))
# print("Validation Set: {} samples".format(len(X_validation)))
# print("Test Set:       {} samples".format(len(X_test)))

In [38]:
# X_train      = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
# X_validation = np.pad(X_validation, ((0,0),(2,2),(2,2),(0,0)), 'constant')
# X_test       = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [39]:
num_classes = 2
dim = 256

directory = "data/womanman-" + str(dim) + "/"
directory_man = "man/"
directory_woman = "woman/"

filelist_man = [file for file in os.listdir(directory + directory_man) if file.endswith('.png')]
filelist_woman = [file for file in os.listdir(directory + directory_woman) if file.endswith('.png')]

randomShuffle = ShuffleSplit(n_splits=10, test_size=.2, random_state=0)

X = np.zeros([len(filelist_man) + len(filelist_woman), dim, dim, 3], dtype=np.uint8)
Y = np.zeros([len(filelist_man) + len(filelist_woman), 1])

accuracies = []

In [40]:
index = 0
for file in filelist_man:
    X[index] = np.array(Image.open(directory + directory_man + file))
    Y[index] = 0
    index = index + 1
for file in filelist_woman:
    X[index] = np.array(Image.open(directory + directory_woman + file))
    Y[index] = 1
    index = index + 1
    
Y = np_utils.to_categorical(Y, 2)

#### Visualize Data

In [41]:
# index = random.randint(0, len(X_train))
# image = X_train[index].squeeze()

# plt.figure(figsize=(1,1))
# plt.imshow(image, cmap="gray")
# print(y_train[index])
# plt.imshow(X[-1])
#print(X.shape)

#### Setupo Tensorflow

In [42]:
EPOCHS = 75
BATCH_SIZE = 128

rate = 0.001

#randomShuffle = ShuffleSplit(n_splits=1, test_size=.2, random_state=0)

In [43]:
def createLeNetModel():
    model = Sequential()

    # Convenlutional layer 1. Input = 32 x 32 x 1. Output = 28 x 28 x 6
    model.add(Conv2D(6, (5, 5), input_shape=(dim, dim, 3), activation='relu'))
    
    # Pooling layer 1. Input = 28 x 28 x 6. Output = 14 x 14 x 6
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    # Convenlutional layer 2. Output = 10 x 10 x 16.
    model.add(Conv2D(16, (5, 5), activation='relu'))
    
    # Pooling layer 2. Input = 10 x 10 x 16. Otuput = 5 x 5 x 16
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))) 
    
    model.add(Flatten())
    
    model.add(Dense(120, activation='relu'))
    
    model.add(Dense(84, activation='relu'))

    # Output layer
    model.add(Dense(2, activation='softmax'))
    
    optimizer_adam = Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer_adam, metrics=['accuracy'])
    print(model.summary())
    return model

In [44]:
for train_index, test_index in randomShuffle.split(X):
    X_train = X[train_index, :]
    Y_train = Y[train_index, :]

    X_test = X[test_index, :]
    Y_test = Y[test_index, :]
    
    model = createLeNetModel()
    
    model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE)
    scores = model.evaluate(X_test, Y_test)
    accuracies.append(scores[1]*100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 252, 252, 6)       456       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 126, 126, 6)       0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 122, 122, 16)      2416      
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 61, 61, 16)        0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 59536)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 120)               7144440   
_________________________________________________________________
dense_59 (Dense)             (None, 84)                10164     
__________

1124/1124 [==============================] - 3s 3ms/step - loss: 0.6267 - acc: 0.8861 - val_loss: 2.0471 - val_acc: 0.6206
Epoch 52/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6457 - acc: 0.8737 - val_loss: 2.0028 - val_acc: 0.6170
Epoch 53/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.6072 - acc: 0.8888 - val_loss: 1.9125 - val_acc: 0.6418
Epoch 54/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5822 - acc: 0.8968 - val_loss: 1.9005 - val_acc: 0.6312
Epoch 55/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5706 - acc: 0.9021 - val_loss: 1.9611 - val_acc: 0.6277
Epoch 56/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5580 - acc: 0.8977 - val_loss: 1.9207 - val_acc: 0.6312
Epoch 57/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5393 - acc: 0.9146 - val_loss: 1.8732 - val_acc: 0.6383
Epoch 58/75
1124/1124 [==============================] - 3s 3ms/ste

Epoch 26/75
1124/1124 [==============================] - 3s 3ms/step - loss: 2.1269 - acc: 0.7544 - val_loss: 3.3081 - val_acc: 0.6064
Epoch 27/75
1124/1124 [==============================] - 3s 3ms/step - loss: 2.0659 - acc: 0.7669 - val_loss: 3.3335 - val_acc: 0.6170
Epoch 28/75
1124/1124 [==============================] - 3s 3ms/step - loss: 2.0167 - acc: 0.7696 - val_loss: 3.3409 - val_acc: 0.6135
Epoch 29/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.9951 - acc: 0.7580 - val_loss: 3.3181 - val_acc: 0.6206
Epoch 30/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.9523 - acc: 0.7669 - val_loss: 3.2739 - val_acc: 0.6135
Epoch 31/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.9062 - acc: 0.7642 - val_loss: 3.2382 - val_acc: 0.6312
Epoch 32/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.8402 - acc: 0.7865 - val_loss: 3.2219 - val_acc: 0.6241
Epoch 33/75
1124/1124 [==============================] 

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 5s 4ms/step - loss: 6.1225 - acc: 0.5089 - val_loss: 5.1338 - val_acc: 0.5035
Epoch 2/75
1124/1124 [==============================] - 3s 3ms/step - loss: 4.9261 - acc: 0.5151 - val_loss: 4.5630 - val_acc: 0.5213
Epoch 3/75
1124/1124 [==============================] - 3s 3ms/step - loss: 4.2301 - acc: 0.5649 - val_loss: 4.3604 - val_acc: 0.5461
Epoch 4/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.8737 - acc: 0.5943 - val_loss: 4.2516 - val_acc: 0.5355
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.6017 - acc: 0.6005 - val_loss: 4.2045 - val_acc: 0.5284
Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.3545 - acc: 0.6201 - val_loss: 4.1131 - val_acc: 0.5319
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.1581 - acc: 0.6343 - val_loss: 4.1240 - val_acc: 0.5390
Epoch 8/75
1124

1124/1124 [==============================] - 3s 3ms/step - loss: 0.8941 - acc: 0.9386 - val_loss: 3.6178 - val_acc: 0.5603
Epoch 62/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8867 - acc: 0.9404 - val_loss: 3.6181 - val_acc: 0.5780
Epoch 63/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8868 - acc: 0.9368 - val_loss: 3.6153 - val_acc: 0.5674
Epoch 64/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8812 - acc: 0.9351 - val_loss: 3.6137 - val_acc: 0.5816
Epoch 65/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8710 - acc: 0.9386 - val_loss: 3.6046 - val_acc: 0.5709
Epoch 66/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8681 - acc: 0.9395 - val_loss: 3.6094 - val_acc: 0.5851
Epoch 67/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.8655 - acc: 0.9395 - val_loss: 3.6075 - val_acc: 0.5851
Epoch 68/75
1124/1124 [==============================] - 3s 3ms/ste

...

Train on 1124 samples, validate on 282 samples
Epoch 1/75
1124/1124 [==============================] - 5s 5ms/step - loss: 6.4693 - acc: 0.4964 - val_loss: 5.9793 - val_acc: 0.4681
Epoch 2/75
1124/1124 [==============================] - 3s 3ms/step - loss: 5.6428 - acc: 0.5036 - val_loss: 5.3655 - val_acc: 0.4716
Epoch 3/75
1124/1124 [==============================] - 3s 3ms/step - loss: 5.0694 - acc: 0.5196 - val_loss: 5.0539 - val_acc: 0.5106
Epoch 4/75
1124/1124 [==============================] - 3s 3ms/step - loss: 4.4937 - acc: 0.5391 - val_loss: 4.4718 - val_acc: 0.5284
Epoch 5/75
1124/1124 [==============================] - 3s 3ms/step - loss: 4.0795 - acc: 0.5560 - val_loss: 4.1665 - val_acc: 0.5638
Epoch 6/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.7711 - acc: 0.5676 - val_loss: 3.8996 - val_acc: 0.5887
Epoch 7/75
1124/1124 [==============================] - 3s 3ms/step - loss: 3.5542 - acc: 0.5792 - val_loss: 3.7437 - val_acc: 0.5993
Epoch 8/75
1124

1124/1124 [==============================] - 3s 3ms/step - loss: 0.5689 - acc: 0.9190 - val_loss: 2.4444 - val_acc: 0.5709
Epoch 62/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5612 - acc: 0.9128 - val_loss: 2.4394 - val_acc: 0.5816
Epoch 63/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5480 - acc: 0.9279 - val_loss: 2.4391 - val_acc: 0.5957
Epoch 64/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5338 - acc: 0.9306 - val_loss: 2.4487 - val_acc: 0.5816
Epoch 65/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5263 - acc: 0.9262 - val_loss: 2.4413 - val_acc: 0.5816
Epoch 66/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5236 - acc: 0.9270 - val_loss: 2.4712 - val_acc: 0.5745
Epoch 67/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.5132 - acc: 0.9288 - val_loss: 2.4570 - val_acc: 0.5887
Epoch 68/75
1124/1124 [==============================] - 3s 3ms/ste

Epoch 36/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.5532 - acc: 0.7625 - val_loss: 3.3499 - val_acc: 0.5922
Epoch 37/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.5002 - acc: 0.7865 - val_loss: 3.2280 - val_acc: 0.6064
Epoch 38/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.5018 - acc: 0.7767 - val_loss: 3.4736 - val_acc: 0.5887
Epoch 39/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.4319 - acc: 0.7980 - val_loss: 3.5938 - val_acc: 0.5887
Epoch 40/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.3895 - acc: 0.8007 - val_loss: 3.3048 - val_acc: 0.5922
Epoch 41/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.3401 - acc: 0.8158 - val_loss: 3.3025 - val_acc: 0.5922
Epoch 42/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.3087 - acc: 0.8230 - val_loss: 3.3257 - val_acc: 0.5851
Epoch 43/75
1124/1124 [==============================] 

1124/1124 [==============================] - 3s 3ms/step - loss: 2.0555 - acc: 0.5943 - val_loss: 2.3626 - val_acc: 0.5567
Epoch 11/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.9206 - acc: 0.5899 - val_loss: 2.3218 - val_acc: 0.5674
Epoch 12/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.8167 - acc: 0.5970 - val_loss: 2.2841 - val_acc: 0.5851
Epoch 13/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.7216 - acc: 0.6103 - val_loss: 2.2444 - val_acc: 0.5957
Epoch 14/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.6376 - acc: 0.6183 - val_loss: 2.1867 - val_acc: 0.5638
Epoch 15/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.5727 - acc: 0.6139 - val_loss: 2.1509 - val_acc: 0.5816
Epoch 16/75
1124/1124 [==============================] - 3s 3ms/step - loss: 1.4855 - acc: 0.6281 - val_loss: 2.1022 - val_acc: 0.5709
Epoch 17/75
1124/1124 [==============================] - 3s 3ms/ste

Epoch 71/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3126 - acc: 0.9039 - val_loss: 1.7461 - val_acc: 0.5745
Epoch 72/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.3058 - acc: 0.9039 - val_loss: 1.7617 - val_acc: 0.5674
Epoch 73/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.2958 - acc: 0.9075 - val_loss: 1.7472 - val_acc: 0.5674
Epoch 74/75
1124/1124 [==============================] - 3s 3ms/step - loss: 0.2958 - acc: 0.9093 - val_loss: 1.7636 - val_acc: 0.5709
Epoch 75/75
282/282 [==============================] - 0s 1ms/step


In [45]:
print(accuracies)

[63.120567629523315, 62.056737504106884, 58.156028622431108, 58.865248480587141, 57.801418651080304, 60.992907674600048, 58.865248226950349, 57.801418355170718, 58.510638551509132, 56.737588906119051]


In [46]:
avg = sum(accuracies) / 10
print(avg)

59.2907802602
